In [3]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
xlsx_path = "../../corpus/NTT/"
xlsx = os.listdir(xlsx_path)[1]

In [3]:
xlsx

'japanese_persona_chat.xlsx'

In [4]:
df = pd.read_excel(xlsx_path+xlsx)
# xlrd version == 1.2.0

In [5]:
"""
    data : name
    convs : [
        "did-num" : []
    ]

"""

def read_dialogue(df):
    convs = dict()
    convs["convs"] = []
    current_id = ""
    conv = []
    for id_, utt in zip(df["ID"], df["発話"]):
        ex_id = id_.split("_")[1]
        # id 変更
        if ex_id != current_id:
            # 最初の制御
            if len(conv) == 0:
                conv.append(utt)
                current_id = ex_id
                continue
            data = {"did-{0}".format(current_id):conv}
            convs["convs"].append(data)
            current_id = ex_id
            conv = []
        conv.append(utt)
    return convs

def read_persona(df):
    convs = dict()
    convs["convs"] = []
    current_id = ""
    conv = []
    for id_, utt in zip(df["No"], df["発話"]):
        # ex_id = id_.split("_")[1]
        # id 変更
        if id_ != current_id:
            # 最初の制御
            if len(conv) == 0:
                conv.append(utt)
                current_id = id_
                continue
            data = {"did-{0}".format(current_id):conv}
            convs["convs"].append(data)
            current_id = id_
            conv = []
        conv.append(utt)
    return convs

# with open("../../corpus/NTT/empathetic.json", "w") as f:
#     json.dump(read_dialogue(df),  f,  ensure_ascii=False, indent=4)

In [6]:
# convs = read_dialogue(df)
convs = read_persona(df)
# with open("../../corpus/NTT/persona.json", "w") as f:
#     json.dump(convs,  f,  ensure_ascii=False, indent=4)

In [7]:
def skip_utt(convs):
    skip_pair = []
    for did in convs["convs"]:
        dids = list( did.keys() )[0]
        conv = did[dids][2::2]
        print(conv)

In [6]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models ,losses
from sentence_transformers.readers import InputExample
from sentence_transformers.losses import TripletDistanceMetric, SoftmaxLoss
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.readers import TripletReader
from sentence_transformers.datasets import SentencesDataset
from torch.utils.data import DataLoader

In [9]:
import transformers
transformers.BertTokenizer = transformers.BertJapaneseTokenizer

In [10]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models

download_path = "../../corpus/"
# download_path = ""
transformer = models.Transformer(download_path+'cl-tohoku/bert-base-japanese-whole-word-masking')
pooling = models.Pooling(transformer.get_word_embedding_dimension(),    
  pooling_mode_mean_tokens=True,
  pooling_mode_cls_token=False, 
  pooling_mode_max_tokens=False
)
model = SentenceTransformer(modules=[transformer, pooling])

sentences = ['吾輩は猫である',  '本日は晴天なり']
embeddings = model.encode(sentences)

for i, embedding in enumerate(embeddings):
  print("[%d] : %s" % (i, embedding.shape, ))

Some weights of the model checkpoint at ../../corpus/cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0] : (768,)
[1] : (768,)


In [11]:
def format_seq(conv):
    if len(conv)%2 != 0:
        conv = conv[:-1]
    return conv

def make_X(convs):
    error_pair = []
    correct_pair = []
    X = []
    for did in convs["convs"]:
        dids = list( did.keys() )[0]
        # 
        conv =  np.array( format_seq( did[dids][2:] ) ).reshape(-1, 2).tolist() 
        # correct_pair += conv
        correct_pair += conv[::2]
        # ----error----
        conv1 =  np.array( format_seq( did[dids][2::2] ) ).reshape(-1, 2).tolist() 
        # conv2 =  np.array( format_seq( did[dids][2::2] ) ).reshape(-1, 2).tolist()
        # error_pair += conv1 + conv2
        error_pair += conv1
        # break
    print("collectted data wrong:{0}, correct:{1}, sum:{2}".format(len(error_pair), len(correct_pair), len(error_pair)+len(correct_pair)))
    # 学習用データ作成
    # 0: not error, 1: error
    for pair in error_pair:
        X.append( InputExample(texts=pair, label=1 ) )  
    for pair in correct_pair:
        X.append( InputExample(texts=pair, label=0 ) )  

    return X

In [12]:
X = make_X(convs)

collectted data wrong:11435, correct:15051, sum:26486


In [13]:
X_train, X_test = train_test_split(X, train_size=0.7, random_state=4)

In [14]:
BATCH_SIZE = 32
NUM_EPOCHS = 3
EVAL_STEPS = 1000
WARMUP_STEPS = int(len(X_train) // BATCH_SIZE * 0.1) 
OUTPUT_PATH = "../../corpus/pretrained/sbert_unclear1"

In [15]:
train_data = SentencesDataset(X_train, model=model)
train_dataloader = DataLoader(train_data, shuffle=True,  batch_size=BATCH_SIZE)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=2)

In [16]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
         epochs=NUM_EPOCHS,
         evaluation_steps=EVAL_STEPS,
         warmup_steps=WARMUP_STEPS,
         output_path=OUTPUT_PATH
         )

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/580 [00:00<?, ?it/s]

Iteration:   0%|          | 0/580 [00:00<?, ?it/s]

Iteration:   0%|          | 0/580 [00:00<?, ?it/s]

In [7]:
from sklearn import svm
# clf = AdaBoostClassifier()
clf = svm.SVC(kernel='rbf', gamma =0.0001, C=1000)

In [8]:
bert_path = "../../corpus/pretrained/sbert_unclear1"
sbert = SentenceTransformer(bert_path)

In [9]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [10]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs_ = read_conv(path, datalist)

In [20]:
def make_Xy_svm(convs):
    errors = ["Topic transition error", "Lack of information", "Unclear intention"]
    errors = errors[0]
    X = []
    y = []
    for conv in convs:
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            if not ut.is_system():
                # システムはエラー
                if conv[i+1].is_exist_error():
                    X.append( [ut.utt, conv[i+1].utt]  )
                    if conv[i+1].is_error_included(errors):
                        y.append(1)
                    else:
                        y.append(0)
    
    return X, y


In [21]:
X_, y_ = make_Xy_svm(convs_)

In [22]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X_, y_, test_size=0.30, random_state=5, stratify=y_)

In [23]:
# X_train = [ text2vec(x).flatten() for x in X_train_str]
# X_test = [ text2vec(x).flatten() for x in X_test_str]

X_train = [ text2feature(x) for x in X_train_str]
X_test = [ text2feature(x)for x in X_test_str]

In [24]:
clf.fit(X_train, y_train)

SVC(C=1000, gamma=0.0001)

In [25]:
y_pred = clf.predict(X_test)

In [26]:
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[225  23]
 [ 31  19]]
accuracy =  0.8187919463087249
precision =  0.4523809523809524
recall =  0.38
f1 score =  0.41304347826086957


- 3エラーすべて

        confusion matrix = 
        [[ 64  42]
        [ 42 150]]
        accuracy =  0.7181208053691275
        precision =  0.78125
        recall =  0.78125
        f1 score =  0.78125

- 発話意図不明確のみ

        confusion matrix = 
        [[115  56]
        [ 69  58]]
        accuracy =  0.5805369127516778
        precision =  0.5087719298245614
        recall =  0.4566929133858268
        f1 score =  0.48132780082987553

        - ほぼ無効

- 情報不足のみ

        confusion matrix = 
        [[279   3]
        [ 16   0]]
        accuracy =  0.9362416107382551
        precision =  0.0
        recall =  0.0
        f1 score =  0.0

        - 想定通りの挙動

- 話題繊維エラーのみ
        
        confusion matrix = 
        [[225  23]
        [ 31  19]]
        accuracy =  0.8187919463087249
        precision =  0.4523809523809524
        recall =  0.38
        f1 score =  0.41304347826086957
        


In [27]:
len(y_test)

298

In [28]:
y_test.count(1)

50